In [4]:
from bs4 import BeautifulSoup
from datetime import datetime, timezone, timedelta
import pandas as pd
import json
import requests
import numpy as np
import re
import time as time
import os
import json
import tensorflow as tf
from tensorflow import keras

In [5]:
np.random.seed(4)
tf.random.set_seed(4)

# Web Scraping
We are going to gather our data while putting minimal pressure on the target website's servers. We will use www.rev.com which has many transcripts available. I will mostly focus on the transcripts of Biden and Trump campaign speeches, before the 2020 election

In [100]:
#Let's give our program some fake credentials to make it appear like a real person is using it
headers= {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.9,fr;q=0.8,ro;q=0.7,ru;q=0.6,la;q=0.5,pt;q=0.4,de;q=0.3',
    'cache-control': 'max-age=0',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36'}

# These are the target url's which are mostly from rallys and speeches
url_list=[
    'https://www.rev.com/blog/transcripts/donald-trump-cpac-2021-speech-transcript',
    'https://www.rev.com/blog/transcripts/donald-trump-speech-transcript-departing-white-house-on-inauguration-day',
    'https://www.rev.com/blog/transcripts/president-donald-trump-farewell-address-speech-transcript',
    'https://www.rev.com/blog/transcripts/donald-trump-second-video-speech-condemning-capitol-violence-transcript',
    'https://www.rev.com/blog/transcripts/donald-trumps-first-comments-since-capitol-riots-says-he-wants-no-violence',
    'https://www.rev.com/blog/transcripts/donald-trump-concedes-election-condemns-rioters-video-transcript-january-7',
    'https://www.rev.com/blog/transcripts/trump-video-telling-protesters-at-capitol-building-to-go-home-transcript',
    'https://www.rev.com/blog/transcripts/donald-trump-speech-save-america-rally-transcript-january-6',
    'https://www.rev.com/blog/transcripts/donald-trump-rally-speech-transcript-dalton-georgia-senate-runoff-election',
    'https://www.rev.com/blog/transcripts/donald-trump-georgia-phone-call-transcript-brad-raffensperger-recording',
    'https://www.rev.com/blog/transcripts/donald-trump-hosts-operation-warp-speed-covid-19-vaccine-summit-transcript-december-8',
    'https://www.rev.com/blog/transcripts/donald-trump-georgia-rally-transcript-before-senate-runoff-elections-december-5',
    'https://www.rev.com/blog/transcripts/donald-trump-speech-on-election-fraud-claims-transcript-december-2',
    'https://www.rev.com/blog/transcripts/donald-trump-remarks-transcript-pennsylvania-republican-hearing-on-2020-election',
    'https://www.rev.com/blog/transcripts/donald-trump-white-house-press-conference-on-lowering-drug-prices-transcript',
    'https://www.rev.com/blog/transcripts/donald-trump-white-house-press-conference-as-election-counts-continue-transcript-november-5',
    'https://www.rev.com/blog/transcripts/donald-trump-rally-speech-transcript-kenosha-wi-november-2',
    'https://www.rev.com/blog/transcripts/donald-trump-rally-speech-transcript-scranton-pa-november-2',
    'https://www.rev.com/blog/transcripts/donald-trump-rally-speech-transcript-fayetteville-nc-november-2',
    'https://www.rev.com/blog/transcripts/donald-trump-rally-speech-transcript-rome-georgia-november-1',
    'https://www.rev.com/blog/transcripts/donald-trump-rally-speech-transcript-waterford-township-michigan-october-30',
    'https://www.rev.com/blog/transcripts/donald-trump-rally-speech-transcript-green-bay-wisconsin-october-30',
    'https://www.rev.com/blog/transcripts/donald-trump-rally-speech-transcript-tampa-fl-october-29',
    'https://www.rev.com/blog/transcripts/donald-trump-rally-speech-transcript-bullhead-city-az-october-28',
    'https://www.rev.com/blog/transcripts/donald-trump-rally-speech-transcript-omaha-ne-october-27',
    'https://www.rev.com/blog/transcripts/donald-trump-rally-speech-transcript-west-salem-wisconsin-october-27',
    'https://www.rev.com/blog/transcripts/donald-trump-rally-speech-transcript-lansing-michigan-october-27',
    'https://www.rev.com/blog/transcripts/donald-trump-circleville-ohio-rally-speech-transcript-october-24'
]

### Our web-scraping function which returns the statements made solely by Donald Trump:

In [70]:
def get_speeches(url_list):
    n=0
    global string_all #A global variable containing all of strings, for greater flexibility in the future.
    string_all=''
    string_Trump=''
    for url in url_list:
        time.sleep(abs(np.random.normal(28,24))) #I don't want to add extra pressure on their servers. I also don't want anyone else running this code to get their IP address blocked, so I
                                                  #added randomness to the timing of our execution via a normal distribution

        html=requests.get('{}'.format(url), headers=headers).text #Get the html data
        soup=BeautifulSoup(html, 'html.parser')

        #Let's now get the main blocks of data which are the transcripts:
        transcript=soup.find_all('p')

        #If the block of text starts with some variation of Donald Trump:, then:
        transcript_DT=[statement for statement in transcript if ('Donald Trump:' in statement.text[:30]) | 
                ('President Trump:' in statement.text[:30]) | ('Donald J. Trump:' in statement.text[:35])] 
        
        for statement in transcript_DT:
            string_Trump+='\n '+re.split('(\n)', statement.text)[-1] #Use regex to split the sentences on \n which proceeds Trump:, retrieve Trump's speech. Example given below
        for statement in transcript:
            string_all+= statement.text
        #To output the progress while loading:
        n+=1
        if n%3==0:
            print('{}% completed'.format( round( 100*((url_list.index(url)+1)/len(url_list)), 1) ))

    return string_Trump

In [1]:
string_Trump=get_speeches(url_list)

In [3]:
#We can also load in the text directly, which I saved from earlier
def load_Trump_str():
    global string_Trump
    string_Trump=''
    with open('text_Donald.txt','r') as file:
        list_Trump=file.readlines()
    for line in list_Trump:
        string_Trump+=line

def write_Trump_str():
    global string_Trump
    with open('text_Donald.txt', 'w') as file:
        file.write(string_Trump)

## Training our RNN on the data
We create a keras Tokenizer which will create a unique number for every word seen in the data. 

In [7]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=False)
tokenizer.fit_on_texts([string_Trump])

In [12]:
tokenizer.texts_to_sequences(["An example"])

[[104, 1048]]

In [13]:
tokenizer.sequences_to_texts([[104, 1048]])

['an example']

In [14]:
max_id = len(tokenizer.word_index) # number of distinct characters
dataset_size=sum(tokenizer.word_counts.values()) #Use for word-level encoding
#dataset_size = tokenizer.document_count # total number of characters. Use for char-level

## Preprocessing Step
In order to train our model, we have to create pairs of sets of words. The first set of every pair will be 100 words long (training set), and the second set will contain one word only. Our model will therefore have to determine that, given an input of 100 words (more words for longer term memory), what it expects the output word to be. We will shuffle our sets of data to create an i.i.d dataset. 100 n_steps represents a fairly long memory on which our model will be trained on.

In [32]:
[encoded] = np.array(tokenizer.texts_to_sequences([string_Trump])) - 1
train_size = dataset_size * 100 // 100 #We will not be using a validation set for the time being!
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

n_steps = 100 #He uses quite lengthy sentences to make his point, so I will set this higher to account for more long term memory
window_length = n_steps + 1 # target = input shifted 1 word ahead
dataset = dataset.repeat().window(window_length, shift=1, drop_remainder=True)

In [33]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))
batch_size = 32
dataset = dataset.shuffle(7600).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [34]:
dataset = dataset.map(lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

In [35]:
dataset = dataset.prefetch(1)

## Model training
We will use a GRU layer instead of LSTM since the GRU is much simpler (and therefore much less memory intensive) and provides very similar precision. Dropout has been shown to improve performance in select cases (especially when using Monte Carlo), and to greatly reduce overfitting. The output will be a softmax function and will have an array of probabilities for each number/word. We will use Adam optimization which uses momentum-like properties to speed up our gradient descent.

In [36]:
first_time=datetime.now()

model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],dropout=0.2),
    keras.layers.GRU(128, return_sequences=True,dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation="softmax"))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, steps_per_epoch=train_size // batch_size,epochs=10)

time_end=datetime.now()-first_time
print(time_end)

In [93]:
#model.save('TrumpBot.h5')